In [ ]:
from cnn_source import Simple3DCNN
import torch
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
squat_model_file_1 = 'model/squat_train.pt'
model_squat_1 = Simple3DCNN().to(device)
model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_location=device))
model_file = 'model/classifying_cnn.pt'
model = Simple3DCNN().to(device)

In [ ]:
import cv2
import numpy as np
import torch

# 실시간 데이터 준비 함수
def prepare_realtime_data(cap, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, C, T, H, W)

    return frames


In [1]:
from cnn_source import Simple3DCNN
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import pickle

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
class_model_file = 'model/classifying_cnn.pt'
model_class = Simple3DCNN().to(device)
model_class.load_state_dict(torch.load(class_model_file,map_location=device))

C:\Users\JPJ\AppData\Local\Temp\ipykernel_24348\1319729835.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_class.load_state_dict(torch.load(class_model_file,map_lo

<All keys matched successfully>

In [3]:
# 데이터 전처리 정의 (프레임 단위로 처리)
transform = transforms.Compose([
    transforms.ToPILImage(),                  # Numpy 배열을 PIL 이미지로 변환          
    transforms.ToTensor(),
    transforms.Resize((640,480))          
])

In [4]:
import cv2
import numpy as np
import torch

# 실시간 데이터 준비 함수
def prepare_realtime_data(cap, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, C, T, H, W)

    return frames

In [5]:
import cv2
import time
import mediapipe as mp
import numpy as np
import torch

# 모델 초기화 (예시)
model = Simple3DCNN().to(device)  # 모델을 디바이스에 맞게 초기화

# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# 웹캡처 객체 생성
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot access the camera.")
    exit()

# 카운트다운 시간 설정
countdown_numbers = [3, 2, 1, "Start"]

# Pose 모델 초기화
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5)

frame_buffer = []  # 마지막 15프레임을 저장할 버퍼

# 카운트다운
for count in countdown_numbers:
    # 1초 동안 텍스트를 표시
    start_time = time.time()
    while time.time() - start_time <= 1:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to capture image.")
            break
        
        # 화면 크기
        height, width, _ = frame.shape
        frame2 = frame.copy()
        # 텍스트 설정
        text = str(count)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 5
        color = (255, 255, 255)

        # 텍스트 중앙 배치
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = (height + text_size[1]) // 2

        # 텍스트 추가
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

        # 화면 표시
        with torch.no_grad():
            while True:
            # 실시간 데이터 준비
                inputs = prepare_realtime_data(cap, frames_per_video=15, device=device)

            # 모델에 입력
                outputs = model(inputs)

        # 출력 처리
                predicted_class = torch.argmax(outputs, dim=1)
                False
        if predicted_class == 0 :
            pose = 'Push'
        elif predicted_class == 1 :                    
            pose = 'squat'
        elif predicted_class == 2 :
            pose = 'lunge'
        else :
            pose = 'pose not included'     
        # 'q'를 누르면 종료
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

# 텍스트 사라진 화면 유지
while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture image.")
        break
    
    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Mediapipe Pose 실행
    results = pose.process(rgb_frame)

    # 랜드마크 그리기
    black_frame = np.zeros_like(frame)  # 검은 화면 생성
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    

    # 모델 입력 차원 맞추기 (T, H, W, C -> T, C, H, W)

    model.eval()    # 모델 예측
    with torch.no_grad():
        while True:
        # 실시간 데이터 준비
            inputs = prepare_realtime_data(cap, frames_per_video=30, height = 480, width =640, device=device)
            # 모델에 입력
            outputs = model(inputs)
        
            # 출력 처리 (e.g., 클래스 예측)
            predicted_class = torch.argmax(outputs, dim=1)
            # 예측 결과 처리
            if predicted_class == 0:
                pose = 'push'
                break
            elif predicted_class == 1:
                pose = 'squat'
                break
            elif predicted_class == 2:
                pose = 'lunge'
                break
            else:
                pose = 'nothing'
                break


    

    # 예측 결과 화면에 표시
    cv2.putText(frame, f"Pose: {pose}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow("camera", frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) == ord('q'):
        break

# 자원 해제
pose.close()
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


AttributeError: 'str' object has no attribute 'process'